In [1]:
sealed trait Animal

final case class Dog(name: String) extends Animal
final case class Cat(name: String) extends Animal

defined trait Animal
defined class Dog
defined class Cat

# Functions variance

Functions are contravariant in their arguments, and covariant in their results. 

Why is this? 

Let's first look at the argument side. If a function accepts a particular type, then it can accept any of it's subtypes. It's similar to making an statement more accurate, by reducing the input space. For example, if I know how to name any `Animal`, I know how to name `Cat`s in particular:

In [2]:
val f1 : Animal => String = _.toString
val f2 : Cat    => String = f1
val f3 : Dog    => String = f1

f1: Animal => String = <function1>
f2: Cat => String = <function1>
f3: Dog => String = <function1>

Now let's look at the result side. If a function returns a specific type, then that result can also be viewed as any of it's **supertypes**. It's similar to reducing the accuracy of a statement by widening the result type. For example, if I know how to clone a `Cat` then I can say I know how to clone _some_ `Animal`:

In [3]:
val f1 : Unit => Cat    = _ => Cat("minino")
val f2 : Unit => Animal = f1

// Same thing happens with other subtypes
val f3 : Unit => Dog    = _ => Dog("pulgas")
val f4 : Unit => Animal = f3

f1: Unit => Cat = <function1>
f2: Unit => Animal = <function1>
f3: Unit => Dog = <function1>
f4: Unit => Animal = <function1>

An important thing to note is that this operations (narrowing the input type, or widening the result type) are safe to do. Transforming a program in such a way won't result in any unexpected runtime error, these are just changes to the specificity of a function.

Summing up the above:
- arguments can be made more specific
- return types can be made more general

# Variance restrictions in Scala

The Scala compiler imposes important restrictions, though. The variance of a type must be preserved: it's not legal to declare a type as covariant and then use it somewhere where it's expected to be contravariant and vice versa. For example:

In [3]:
type MyContra1[-X] = X
type MyCov1[+Y] = MyContra1[Y] // Y has been declared covariant, it's not possible to pass it to `MyContra1`

cmd3.sc:1: contravariant type X occurs in covariant position in type [-X]X of type MyContra1
type MyContra1[-X] = X
     ^

: 

In [3]:
type MyCov2[+X] = X
type MyContra2[-Y] = MyCov2[Y] // Y has been declared contravariant, it's not possible to pass it to `MyCov2`

cmd3.sc:2: contravariant type Y occurs in covariant position in type [-Y]Helper.this.MyCov2[Y] of type MyContra2
type MyContra2[-Y] = MyCov2[Y] // Y has been declared contravariant, it's not possible to pass it to `MyCov2`
     ^

: 

# Variance restrictions in functions

Because of the covariance restrictions over functions you cannot put a covariant type in as an argument:

In [3]:
trait Vet[+T] {
  def heal(animal: T): Boolean
}

cmd3.sc:2: covariant type T occurs in contravariant position in type T of value animal
  def heal(animal: T): Boolean
           ^

: 

Important to note is that this error message is very general, it doesn't say it's specifically related to functions.

Similarly, you cannot put a contravariant type as a result:

In [3]:
trait Lab[-T] {
  def cloneAnimal(): T
}

cmd3.sc:2: contravariant type T occurs in covariant position in type ()T of method cloneAnimal
  def cloneAnimal(): T
      ^

: 

Why these restrictions?

First, a mental exercise for covariance:

In [4]:
trait Vet[+T] {
   def heal(animal: Any): Boolean // IMAGINE it's not Any but T
}

// This widening works because of covariance: Cat <: Animal => Vet[Cat] <: Vet[Animal]
val notReallyAGeneralVet: Vet[Animal] = new Vet[Cat] {
     def heal(animal: Any): Boolean = { // IMAGINE it's not Any but Cat
       animal match {
         case Cat(_) => 
            true
         case other =>
            throw new Exception(s"Oh not, got an unexpected ${other.getClass}")
       }
     }
} 

val dog: Dog = Dog("Pulgas")

notReallyAGeneralVet.heal(dog) // this invocation, if it had compiled with a generic type T, would be unsafe.

: 

Allowing covariance in arguments makes a method **forget** the specific type it works with, that's why it's not permitted. It allows us to "lie": we can turn a method that only receives a _specific subset_ of values into one that receives a _more general set_ of values.

In the example above, covariance in the argument would have made a function that only works accepting `Cats` be allowed to be invoked with _an_ `Animal`, in particular one that's not a `Cat`, like a `Dog`. This is an error that the variance annotations can help catch at compile time.

Now, a mental exercise with contravariance:

In [5]:
trait Lab[-T] {
  def cloneAnimal(): Any // IMAGINE it's not Any but T
}

// This narrowing works because of contravariance: Cat <: Animal => Lab[Animal] <: Lab[Cat]
val notReallyACatCloner: Lab[Cat] = new Lab[Animal] {
  def cloneAnimal(): Any = // IMAGINE it's not Any but Animal
     Dog("pulgas")
}

// cloneAnimal should return a Cat, because it's type is Lab[Cat].
val cat = notReallyACatCloner.cloneAnimal().asInstanceOf[Cat]

: 

Similarly, allowing contravariance in return types allows making 

In [6]:
// A = Cat
// A1 >: Cat => Animal
sealed trait List[+A] {
  def prepend[A1 >: A](other: A1): List[A1] = ???
}
case class Cons[A](head: A, tail: List[A]) extends List[A]
case object Empty extends List[Nothing]

val y: List[Cat]    = Cons(Cat("c"), Empty)
val x: List[Animal] = y

defined trait List
defined class Cons
defined object Empty
y: List[Cat] = Cons(Cat(c),Empty)
x: List[Animal] = Cons(Cat(c),Empty)

In [10]:
// trait VetC[-A] {
//  def rescueAnimal[B <: A]():B = ???
// }
// val vet: VetC[Animal] = new VetC[Animal] {}
// lazy val dog: Dog = vet.rescueAnimal()

trait Sink[-T] {
 def process(t: T): Unit
}

trait SinkSink[-T] {
 def process(sink: T): Unit
}

defined trait Sink
defined trait SinkSink